# Rejection of ambiguous minimum

### Optional part (2)

In this file we tried to improve the performance of our baseline algorithm discarding the matches when the SAD provided an ambiguous minimum. The data reported in the corresponding section of the report refer to these results.

In [ ]:
import numpy as np
import cv2
from matplotlib import pyplot as plt


In [ ]:
def open_img(img):
    image=cv2.imread(img, cv2.IMREAD_GRAYSCALE)
    copied_image= np.copy(image)
    plt.imshow(copied_image, cmap='gray', vmin=0, vmax=255)
    plt.show()
    
    return copied_image


#functions for SAD implementation
def AD_left(imgL, imgR, i, j, disp, m, n):
    S=np.abs(int(imgL[i+m, j+n])-int(imgR[i+m, j+n+disp]))
    
    return S


#function to calculate the mapping error taking into account invalid pixels
def Error2( target, img, w, h, m, scale_factor, v_pixel, d_max, ig_border):
    error=0
    for j in range(m+d_max, w-m-d_max-ig_border):
        for i in range(m, h-m-ig_border):
            if(img[i,j]!=255):
                if(abs(target[i, j]/scale_factor - img[i, j]/scale_factor)>1):
                    error=error+1
    
    return (error/v_pixel)

The following lines implement the loading of the various images the dataset provided. As a consequence, please gradually markdown the blocks in succession, in orther to work only with the disired image each time.

###### Map

In [ ]:
mapL= open_img("map/im1.pgm")
mapR= open_img("map/im0.pgm")
mapG= open_img("map/disp1.pgm")

#parameters
disp_min =0
disp_max =29
disp_scale =8
ignore_border =0

#image shape values
img_raw=mapL.shape[0]
img_column=mapL.shape[1]

imageL=mapL
imageR=mapR

###### Sawtooth

sawtoothL= open_img("sawtooth/im6.ppm")
sawtoothR= open_img("sawtooth/im2.ppm")
sawtoothG= open_img("sawtooth/disp6.pgm")

disp_min =0
disp_max =19
disp_scale= 8
ignore_border= 0

#image shape values
img_raw=sawtoothL.shape[0]
img_column=sawtoothL.shape[1]

imageL=sawtoothL
imageR=sawtoothR

###### Tsukuba

tsukubaL= open_img("tsukuba/scene1.row3.col3.ppm")
tsukubahR= open_img("tsukuba/scene1.row3.col2.ppm")
tsukubaG= open_img("tsukuba/truedisp.row3.col3.pgm")

disp_min =0
disp_max =15
disp_scale= 16
ignore_border= 18

#image shape values
img_raw=tsukubaL.shape[0]
img_column=tsukubaL.shape[1]

imageL=tsukubaL
imageR=tsukubahR

###### Venus

venusL= open_img("venus/im6.ppm")
venusR= open_img("venus/im2.ppm")
venusG= open_img("venus/disp6.pgm")

disp_min=0
disp_max =19
disp_scale= 8

ignore_border= 0

img_raw=venusL.shape[0]
img_column=venusL.shape[1]

imageL=venusL
imageR=venusR

Now the supporting window'size is chosen. As we have explained in the report, we have tried multiple values, seeking for the better one that fits each image. According to our experimental results K=3 best fits Map and Sawtooth, while K=4 should be selected for Tsukuba and Venus.

In [ ]:
#window shape
K=3

MapL=np.zeros((img_raw, img_column), dtype=np.uint8)

The algorithm used is identical to the base one until the saving of the disparity values calculated for a pixel within the disparity range. Then, these values are sorted in ascending order and the match is considered valid if the difference of the first element of the vector (minimum) with the third one is greater than a certain threshold, set at 10%.

In [ ]:
#in order to get every pixels' position in the image we compute two loops 
disp=0
valid_pixel=0
for i in range(K, img_raw-K-ignore_border):
    for j in range(K+disp_max, img_column-K-ignore_border-disp_max): 
        #We select the pixel and move along its raw
        min_SAD=500000
       
        #moving along the epipolar line
        x=0
        sad_array=np.zeros(disp_max+1-disp_min)
        for d in range(disp_min, disp_max+1):
            SAD=0
            for m in range(-K, K+1):
                for n in range(-K, K+1):
                    SAD=SAD+AD_left(imageL, imageR, i, j, d, m, n)
                    
            #for each disparity value save the SAD obtained in an array
            sad_array[x]=SAD
            x=x+1
            
            if(SAD<=min_SAD):
                min_SAD=SAD
                disp=d 
                
        #sort the array and mark the match as invalid (white) if the minimum is not sharp enough
        sad_array=np.sort(sad_array)
        if ((sad_array[2]-sad_array[0])<0.1*sad_array[0]):
            disp=255/disp_scale
        else:
            valid_pixel=valid_pixel+1
            
        
        MapL[i,j]=np.asarray([disp*disp_scale])

print("Left map")
plt.imshow(MapL, cmap='gray', vmin=0, vmax=255)
plt.show()

total_pixel= (img_column-ignore_border-2*disp_max-2*K)*(img_raw-ignore_border-2*K)
print("percentage of invalid pixels in the image", 1-(valid_pixel/total_pixel))

The mapping error is computed. Again, as now not all the pixels feature a valid disparity value, the standard Error function has been modified in order to take that into account. The line of interest should be uncommented.

In [ ]:
mapping_error=Error2(mapG,MapL,img_column,img_raw,K, disp_scale, valid_pixel, disp_max,ignore_border)
#mapping_error=Error2(sawtoothG,MapL,img_column,img_raw,K, disp_scale, valid_pixel, disp_max, ignore_border)
#mapping_error=Error2(tsukubaG,MapL,img_column,img_raw,K, disp_scale,valid_pixel, disp_max, ignore_border)
#mapping_error=Error2(venusG,MapL,img_column,img_raw,K, disp_scale, valid_pixel, disp_max, ignore_border)

print("the mapping error is:\n", mapping_error)

If desired, the obtained disparity map could be saved uncommenting and executing the following code.

In [ ]:
#cv2.imwrite("disparity_map_minimum.png", MapL)
#cv2.imwrite("disparity_sawtooth_minimum.png", MapL)
#cv2.imwrite("disparity_tsukuba_minimum.png", MapL)
#cv2.imwrite("disparity_venus_minimum.png", MapL)